<a href="https://colab.research.google.com/github/SaulHL08/Notebooks-Datos-Masivos/blob/main/FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import cv2
import os

# Rutas de los videos en Google Drive
video_paths = [
    '/content/drive/MyDrive/FaceRecogTrain/Joshua.mp4',
    '/content/drive/MyDrive/FaceRecogTrain/Luis.mp4',
    '/content/drive/MyDrive/FaceRecogTrain/Marco.mp4',
    '/content/drive/MyDrive/FaceRecogTrain/Saul.mp4'
]

# Nombres de las carpetas de salida
output_directories = [
    'video1_frames/',
    'video2_frames/',
    'video3_frames/',
    'video4_frames/'
]

# Crea los directorios si no existen
for directory in output_directories:
    os.makedirs(directory, exist_ok=True)

# Procesa cada video
for video_path, output_directory in zip(video_paths, output_directories):
    # Lee el video
    cap = cv2.VideoCapture(video_path)

    # Inicializa un contador de frames
    frame_count = 0

    while(cap.isOpened()):
        # Lee el frame actual
        ret, frame = cap.read()

        if ret:
            # Guarda el frame como una imagen
            frame_path = os.path.join(output_directory, f'frame_{frame_count:04d}.jpg')
            cv2.imwrite(frame_path, frame)
            frame_count += 1
        else:
            break

    # Libera el objeto VideoCapture
    cap.release()

    print(f'Se han guardado {frame_count} frames en {output_directory}')

# Cierra todas las ventanas
cv2.destroyAllWindows()



Se han guardado 1241 frames en video1_frames/
Se han guardado 1264 frames en video2_frames/
Se han guardado 1240 frames en video3_frames/
Se han guardado 1413 frames en video4_frames/


In [ ]:
import shutil

# Comprime la carpeta
shutil.make_archive("/content/Nombre_Carpeta", 'zip', "/content/Nombre_Carpeta")


'/content/Nombre_Carpeta.zip'

In [ ]:
import cv2

img = cv2.imread('/content/video4_frames/frame_0909.jpg', cv2.IMREAD_UNCHANGED)

print('Original Dimensions : ', img.shape)

width = int(img.shape[1] * 28 / 100)
height = int(img.shape[0] * 26 / 100)
dim = (width, height)

resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)

cv2.imwrite('/content/video4_frames/frame_0909_resized.jpg', resized)

Original Dimensions :  (1080, 1920, 3)


True

In [ ]:
from os import listdir
from os.path import isfile, join

onlyfiles = [ f for f in listdir(pathFiles) if isfile(join(pathFiles,f)) ]

images = numpy.empty(len(onlyfiles), dtype=object)

  for n in range(0, len(onlyfiles)):
    #     print(join(pathFiles,onlyfiles[n]))
    images[n] = cv2.imread(join(pathFiles,onlyfiles[n]) )

In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Paso 1: Cargar las imágenes y etiquetas


dataset_dir = "/content/Dataset"
personas = ["Joshua", "Luis", "Saul", "Marco"]
etiquetas = []
imagenes = []

for i, persona in enumerate(personas):
    carpeta_persona = os.path.join(dataset_dir, persona)
    for archivo in os.listdir(carpeta_persona):
        ruta_imagen = os.path.join(carpeta_persona, archivo)
        imagen = cv2.imread(ruta_imagen)
        imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
        imagen = cv2.resize(imagen, (537, 280))  # Ajusta el tamaño según sea necesario
        imagenes.append(imagen)
        etiquetas.append(i)

imagenes = np.array(imagenes) / 255.0  # Normaliza los valores de píxeles
etiquetas = np.array(etiquetas)

# Paso 2: Dividir el dataset en entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas, test_size=0.2, random_state=42)

# Paso 3: Definir y compilar el modelo

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(537, 280, 1)),  # Tamaño ajustado a las imágenes
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')  # 4 para las 4 personas
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Paso 4: Entrenar el modelo

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Paso 5: Evaluar el modelo

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_acc}')


In [ ]:
from tensorflow.keras.models import save_model

save_model(model, "/content/drive/MyDrive/models/modelo_reconocimiento_facial.h5")

<ipython-input-6-8654b7022e35>:3: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, "/content/drive/MyDrive/models/modelo_reconocimiento_facial.h5")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
model = load_model("/content/drive/MyDrive/models/modelo_reconocimiento_facial.h5")

# Cargar y preprocesar la imagen
ruta_imagen_nueva = "/content/PXL_20231109_213404147.jpg"  # Reemplaza con la imagen
imagen_nueva = cv2.imread(ruta_imagen_nueva)
imagen_nueva = cv2.cvtColor(imagen_nueva, cv2.COLOR_BGR2GRAY)  # Convertir a escala de grises
imagen_nueva = cv2.resize(imagen_nueva, (100, 100))  # Ajusta el tamaño según sea necesario
imagen_nueva = np.array(imagen_nueva) / 255.0  # Normaliza los valores de píxeles
imagen_nueva = np.expand_dims(imagen_nueva, axis=0)  # Añade una dimensión adicional para el lote

# Realizar la predicción
prediccion = model.predict(imagen_nueva)

# Obtener la etiqueta predicha
etiqueta_predicha = np.argmax(prediccion)

# Asignar nombres a las etiquetas
nombres_personas = ["Joshua", "Luis", "Saul", "Marco"]
nombre_predicho = nombres_personas[etiqueta_predicha]

print(f"La imagen pertenece a: {nombre_predicho}")


1/1 [==============================] - 0s 79ms/step
La imagen pertenece a: Marco
